In [12]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import pickle
import os
import argparse
from sklearn.preprocessing import MinMaxScaler
from sksurv.ensemble import GradientBoostingSurvivalAnalysis, ComponentwiseGradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored, integrated_brier_score, brier_score, concordance_index_ipcw
from sksurv.functions import StepFunction
from sksurv.nonparametric import kaplan_meier_estimator
import matplotlib.ticker as mtick
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [2]:
# 导入功能模块
from func.data_utils import load_data, load_variable_definitions, load_lasso_features, prepare_data
from func.data_utils_test import load_data_test, prepare_data_test


In [7]:

seed, predictor, output_dir = 3456, "pro", "/home/louchen/UKB_meno_pre/s2_model"
print(f"配置: seed={seed}, predictor={predictor}", flush=True)

配置: seed=3456, predictor=pro


In [8]:
# 数据准备
print("数据加载中...", flush=True)
surv_pred_train = load_data(seed)
surv_pred_test = load_data_test(seed)
exp_var, out_surv, pro_var = load_variable_definitions()
pro_lasso = load_lasso_features(seed)
X_train_model, Y_train, groups_train = prepare_data(surv_pred_train, pro_lasso, out_surv, predictor=predictor, exp_var=exp_var)
X_test_model, Y_test, groups_test = prepare_data_test(surv_pred_test, pro_lasso, out_surv, predictor=predictor, exp_var=exp_var)

print(f"数据准备完成: {predictor}类型, 训练集{X_train_model.shape[1]}个特征, {X_train_model.shape[0]}个样本", flush=True)
print(f"测试集: {X_test_model.shape[1]}个特征, {X_test_model.shape[0]}个样本", flush=True)

数据加载中...
数据准备完成: pro类型, 训练集166个特征, 2822个样本
测试集: 166个特征, 940个样本


In [10]:
# Create output directory for results
os.makedirs(output_dir, exist_ok=True)

# Load the trained model with seed and predictor
model_dir = os.path.join(output_dir, f"model/{predictor}/gbm")
model_path = os.path.join(model_dir, f"gbm_best_model_{seed}.pkl")
with open(model_path, "rb") as f:
    loaded_model = pickle.load(f)

print(f"加载模型: {model_path}")

加载模型: /home/louchen/UKB_meno_pre/s2_model/model/pro/gbm/gbm_best_model_3456.pkl


In [15]:
# Predict risk scores on test data
risk_scores = loaded_model.predict(X_test_model)
c_index_result = concordance_index_censored(
    Y_test['e.tdm'],
    Y_test['t.tdm'],
    risk_scores
)
c_index = c_index_result[0]
print(f"C-index (测试集): {c_index:.4f}")

C-index (测试集): 0.6901


In [41]:
lower, upper = np.percentile(Y_test["t.tdm"], [10, 90])
print(lower,upper)
brier_times = np.arange(lower, upper + 1)
brier_times

1696.4 5394.4


array([1696.4, 1697.4, 1698.4, ..., 5392.4, 5393.4, 5394.4])

In [57]:
surv_prob = np.vstack([fn(brier_times) for fn in loaded_model.predict_survival_function(X_test_model)])


In [45]:
random_surv_prob = 0.5 * np.ones((Y_test.shape[0], brier_times.shape[0]))

In [47]:
km_func = StepFunction(*kaplan_meier_estimator(Y_test["e.tdm"], Y_test["t.tdm"]))
km_surv_prob = np.tile(km_func(brier_times), (Y_test.shape[0], 1))

In [48]:
score_brier = pd.Series(
    [
        integrated_brier_score(Y_train, Y_test, prob, brier_times)
        for prob in (surv_prob, random_surv_prob, km_surv_prob)
    ],
    index=["GBM", "Random", "Kaplan-Meier"],
    name="IBS",
)

score_brier

GBM             0.129862
Random          0.252032
Kaplan-Meier    0.130013
Name: IBS, dtype: float64

In [ ]:
survs = loaded_model.predict_survival_function(X_test_model)
preds = [fn(365.25*3) for fn in survs]
times, score = brier_score(Y_train, Y_test, preds, 365.25*3)
print(score)

In [63]:
survs = loaded_model.predict_survival_function(X_test_model)
preds = [fn(365.25*5) for fn in survs]
times, score = brier_score(Y_train, Y_test, preds, 365.25*5)
print(score)

[0.08758378]


In [64]:
survs = loaded_model.predict_survival_function(X_test_model)
preds = [fn(365.25*10) for fn in survs]
times, score = brier_score(Y_train, Y_test, preds, 365.25*10)
print(score)

[0.13805849]
